In [38]:
project_pth="/home/ckq/CHATGPT/niuren/InstructGLM"
old_glm_path="/home/ckq/CHATGPT/0model_pretrained/chatglm_3_30"

In [39]:
from transformers import HfArgumentParser
import torch
import transformers
from peft import get_peft_model, LoraConfig, TaskType
from dataclasses import dataclass, field

In [40]:
from transformers import AutoTokenizer, AutoModel, TrainingArguments, AutoConfig
from modeling_chatglm import ChatGLMForConditionalGeneration
import torch
import torch.nn as nn
from peft import get_peft_model, LoraConfig, TaskType


torch.set_default_tensor_type(torch.cuda.HalfTensor)
model = ChatGLMForConditionalGeneration.from_pretrained(old_glm_path, trust_remote_code=True, device_map='auto')

The argument `trust_remote_code` is to be used with Auto classes. It has no effect here and is ignored.
Loading checkpoint shards: 100%|██████████| 8/8 [00:09<00:00,  1.18s/it]


In [41]:

peft_path = project_pth+"/output/alpaca/chatglm-lora.pt"

peft_config = LoraConfig(
    task_type=TaskType.CAUSAL_LM, inference_mode=False,
    r=8,
    lora_alpha=32, lora_dropout=0.1
)

model = get_peft_model(model, peft_config)
model.load_state_dict(torch.load(peft_path), strict=False)
torch.set_default_tensor_type(torch.cuda.FloatTensor)

In [ ]:

peft_path = "./output/alpaca/chatglm-lora.pt"

peft_config = LoraConfig(
    task_type=TaskType.CAUSAL_LM, inference_mode=False,
    r=8,
    lora_alpha=32, lora_dropout=0.1
)

model = get_peft_model(model, peft_config)
model.load_state_dict(torch.load(peft_path), strict=False)
torch.set_default_tensor_type(torch.cuda.FloatTensor)

In [42]:
tokenizer = AutoTokenizer.from_pretrained("THUDM/chatglm-6b", trust_remote_code=True)

Explicitly passing a `revision` is encouraged when loading a model with custom code to ensure no malicious code has been contributed in a newer revision.


In [43]:
import json
from tqdm import tqdm

instructions = json.load(open(project_pth+"/data/alpaca_data.json"))

instructions = [
    {
        'instruction': "下面哪个产品与其他不同",
        "input": "知乎, 百度, 微博, 拼多多",
        "output": "拼多多与其他产品不同，因为它是一个电子商务平台，而知乎、百度和微博都是社交平台/搜索引擎。",
    },
    {
        'instruction': "下面哪个产品与其他不同",
        "input": "chatgpt, 文心, CPM, 拼多多",
        "output": "拼多多与其他产品不同，因为它是一家电子商务公司，而ChatGPT、文心和CPM都不是电子商务公司。",
    }
]

In [44]:
answers = []

with torch.no_grad():
    for idx, item in enumerate(instructions[:5]):
        input_text = f"### {idx+1}.Instruction:\n{item['instruction']}\n\n"
        if item.get('input'):
            input_text += f"### {idx+1}.Input:\n{item['input']}\n\n"
        input_text += f"### {idx+1}.Response:"
        batch = tokenizer(input_text, return_tensors="pt")
        out = model.generate(
            input_ids=batch["input_ids"],
            attention_mask=torch.ones_like(batch["input_ids"]).bool(),
            max_length=512,
            temperature=0
        )# chatGLM报错
        out_text = tokenizer.decode(out[0])
        answer = out_text.replace(input_text, "").replace("\nEND", "").strip()
        item['infer_answer'] = answer
        print(out_text)
        print(f"### {idx+1}.Answer:\n", item.get('output'), '\n\n')
        answers.append({'index': idx, **item})

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:10                                                                                   │
│                                                                                                  │
│    7 │   │   │   input_text += f"### {idx+1}.Input:\n{item['input']}\n\n"                        │
│    8 │   │   input_text += f"### {idx+1}.Response:"                                              │
│    9 │   │   batch = tokenizer(input_text, return_tensors="pt")                                  │
│ ❱ 10 │   │   out = model.generate(                                                               │
│   11 │   │   │   input_ids=batch["input_ids"],                                                   │
│   12 │   │   │   attention_mask=torch.ones_like(batch["input_ids"]).bool(),                      │
│   13 │   │   │   max_length=512,                                                                 │
│                                                                                                  │
│ /home/ckq/anaconda3/envs/accel/lib/python3.9/site-packages/peft/peft_model.py:716 in generate    │
│                                                                                                  │
│    713 │   │   self.base_model.prepare_inputs_for_generation = self.prepare_inputs_for_generati  │
│    714 │   │   try:                                                                              │
│    715 │   │   │   if not isinstance(peft_config, PromptLearningConfig):                         │
│ ❱  716 │   │   │   │   outputs = self.base_model.generate(**kwargs)                              │
│    717 │   │   │   else:                                                                         │
│    718 │   │   │   │   if "input_ids" not in kwargs:                                             │
│    719 │   │   │   │   │   raise ValueError("input_ids must be provided for Peft model generati  │
│                                                                                                  │
│ /home/ckq/anaconda3/envs/accel/lib/python3.9/site-packages/torch/utils/_contextlib.py:115 in     │
│ decorate_context                                                                                 │
│                                                                                                  │
│   112 │   @functools.wraps(func)                                                                 │
│   113 │   def decorate_context(*args, **kwargs):                                                 │
│   114 │   │   with ctx_factory():                                                                │
│ ❱ 115 │   │   │   return func(*args, **kwargs)                                                   │
│   116 │                                                                                          │
│   117 │   return decorate_context                                                                │
│   118                                                                                            │
│                                                                                                  │
│ /home/ckq/anaconda3/envs/accel/lib/python3.9/site-packages/transformers/generation/utils.py:1437 │
│ in generate                                                                                      │
│                                                                                                  │
│   1434 │   │   │   │   )                                                                         │
│   1435 │   │   │                                                                                 │
│   1436 │   │   │   # 11. run greedy search                                                       │
│ ❱ 1437 │   │   │   return self.greedy_search(                                                    │
│   1438 │   │   │   │   input_ids,                          

: 